In [1]:
from nltk.corpus.reader.wordnet import Lemma
import nltk
from nltk import FreqDist
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
# For file reading
import os
from os import listdir
# For pre-processing 
from nltk import word_tokenize
from nltk.tokenize import punkt
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
import string
import time

from collections import Counter
import numpy as np
from sklearn.cluster import KMeans


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rubyli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/rubyli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/rubyli/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Task 1 - Step 1 - Pre-processing words and get top 50 frequent words

In [2]:
def read_data(path: str) -> list:
    """ Read files from a directory and then append the data of each file into a list. """
    folder = listdir(path)
    res = []
    for files in folder:
        # check if current path is a file
        if files != "README.txt":
            filePath = os.path.join(path, files)
            if os.path.isfile(filePath):
                with open(filePath, "r") as file:
                    lines = file.readlines()
                res.append(lines)
    return res

In [3]:
path = "/Users/rubyli/Desktop/GitHubRepos/UoM/NLP-Distributional-Semantics-Neural-Network-for-Classifying-Product-Reviews/product_reviews"
res = read_data(path)
doc = ""
for a in res:
    for b in a:
        doc += b
print(res[0][:5])
# print(doc)

['[t]\n', 'Works[+3]##Works great, no odor, and uses regular bags.\n', "##Can't complain at all!\n", '[t]\n', "Diaper Champ[+2]##So far (3 weeks), we've had no problems with the Diaper Champ at all.\n"]


In [4]:
from nltk import word_tokenize
def process_document(document: str) -> list:
        """ pre-process a document and return a list of its terms: str->list"""
        
        # Remove number
        text_nonum = re.sub(r'\d+', ' ', document)
        
        pattern = r'''(?x)        # set flag to allow verbose regexps
                    (?:[A-Z]\.)+     #abbreviations
                    |\[
                    |[^\w\s]
                    |\#
                    |[-.(]+           #double hyphen, ellipsis, open parenthesis
                    |\S\w*
                    |\$?\d+(?:\.\d+)?%? #currency and percentages
        '''
        #Tokenization
        tokenList = nltk.regexp_tokenize(text_nonum, pattern)
        #To lower case
        tokenList = [word.lower() for word in tokenList]
        #Remove Punctuation
        tokenList = list(filter(lambda word: punkt.PunktToken(word).is_non_punct,tokenList))
        #Remove stopwords
        stopW = stopwords.words("english")
        stopW.append("u")
        stopW.append("p")
        # stopW.append("mp")
        tokenList = list(filter(lambda word: word not in stopW,tokenList))
        # Lemmatisation 
        lemma = WordNetLemmatizer()
        tokenList = [lemma.lemmatize(word) for word in tokenList]

        return tokenList 
    
#     text = document.lower()
    
#     # remove numbers
#     text_nonum = re.sub(r'\d+', ' ', text)
    
#     text_p = "".join([char for char in text_nonum if char not in string.punctuation])
    
#     words = word_tokenize(text_p)
    
#     stop_words = stopwords.words('english')
#     stop_words.append("im")
#     stop_words.append("ive")
    
#     filtered_words = [word for word in words if word not in stop_words]
    
#     lemma = WordNetLemmatizer()
#     tokenList = [lemma.lemmatize(word) for word in filtered_words]
    
#     return tokenList
    

In [5]:
producedDoc = process_document(doc)
print(producedDoc[:50])

['work', 'work', 'great', 'odor', 'us', 'regular', 'bag', 'complain', 'diaper', 'champ', 'far', 'week', 'problem', 'diaper', 'champ', 'diaper', 'contains', 'smell', 'baby', 'diaper', 'use', 'kind', 'bag', 'inside', 'also', 'sprinkled', 'baking', 'soda', 'bottom', 'diaper', 'champ', 'help', 'absorb', 'odor', 'every', 'awhile', 'empty', 'old', 'baking', 'soda', 'replace', 'odor', 'refill', 'know', 'run', 'trouble', 'road', 'odor', 'far', 'complaint']


In [6]:
# Find the 50 most frequently occurred words
# Get the frequency of each word
word_frequencies = FreqDist(producedDoc)
word_frequencies

FreqDist({'use': 353, 'phone': 351, 'one': 337, 'router': 337, 'ipod': 329, 'camera': 322, 'player': 313, 'get': 274, 'battery': 264, 'diaper': 231, ...})

In [7]:
# Sort the dictionary by frequency
sorted_frequencies = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)

# Select the top 50 words
target_words = [item[0] for item in sorted_frequencies[:50]]
print(target_words)

['use', 'phone', 'one', 'router', 'ipod', 'camera', 'player', 'get', 'battery', 'diaper', 'product', 'work', 'like', 'great', 'time', 'feature', 'problem', 'good', 'quality', 'zen', 'would', 'also', 'sound', 'computer', 'software', 'picture', 'well', 'really', 'micro', 'take', 'easy', 'thing', 'even', 'first', 'used', 'need', 'creative', 'bag', 'much', 'want', 'better', 'champ', 'mp', 'look', 'go', 'size', 'music', 'norton', 'little', 'price']


# Task 1 - Step 2 - Sample and Pseudowords

In [8]:
import random

# Sample half of the target words
sample_size = len(target_words) // 2
sample = random.sample(target_words, sample_size)
# Create pseudowords for the sampled target words
madeups = [word[::-1] for word in sample]
# Replace the sampled occurrences of the target words with their pseudowords
pseudowords = target_words[:]
for i, word in enumerate(target_words):
    if word in sample:
        # Find the index of the word to be replace
        replacement_index = sample.index(word)
        # Replace the word with the corresponding word from the replacement list
        pseudowords[i] = madeups[replacement_index]

In [9]:
print(pseudowords)

['esu', 'enohp', 'eno', 'router', 'ipod', 'camera', 'reyalp', 'teg', 'battery', 'diaper', 'tcudorp', 'work', 'like', 'taerg', 'emit', 'feature', 'melborp', 'good', 'quality', 'zen', 'dluow', 'also', 'sound', 'retupmoc', 'software', 'erutcip', 'llew', 'yllaer', 'micro', 'take', 'ysae', 'gniht', 'neve', 'first', 'used', 'deen', 'creative', 'gab', 'hcum', 'want', 'retteb', 'pmahc', 'mp', 'look', 'og', 'ezis', 'music', 'norton', 'little', 'ecirp']


# Task 1 - Step 3 - Feature Vector

In [10]:
# Create a corpus containing the target words and pseudowords
corpus = target_words + pseudowords
print(len(corpus))

100


In [11]:
def word_count(str):
    counts = dict()
    words = str.split()
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

In [12]:
# Get all the sentences in all documents
sentences = []
for doc in res:
    for sen in doc:
        sentences.append(sen)

print(sentences[:4])

['[t]\n', 'Works[+3]##Works great, no odor, and uses regular bags.\n', "##Can't complain at all!\n", '[t]\n']


In [13]:
# Loop through each sentence and count the frequency of each word
from collections import Counter

sen_count = []
for i, sentence in enumerate(sentences):
    counts = dict()
    words = process_document(sentence)
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    sen_count.append(counts)
print(sen_count[:10])

[{}, {'work': 2, 'great': 1, 'odor': 1, 'us': 1, 'regular': 1, 'bag': 1}, {'complain': 1}, {}, {'diaper': 2, 'champ': 2, 'far': 1, 'week': 1, 'problem': 1}, {'diaper': 2, 'contains': 1, 'smell': 1, 'baby': 1, 'use': 1, 'kind': 1, 'bag': 1, 'inside': 1}, {'also': 1, 'sprinkled': 1, 'baking': 2, 'soda': 2, 'bottom': 1, 'diaper': 1, 'champ': 1, 'help': 1, 'absorb': 1, 'odor': 1, 'every': 1, 'awhile': 1, 'empty': 1, 'old': 1, 'replace': 1}, {'odor': 2, 'refill': 2, 'know': 1, 'run': 1, 'trouble': 1, 'road': 1, 'far': 1, 'complaint': 1, 'happy': 1, 'buy': 1}, {}, {'started': 1, 'diaper': 1, 'genie': 1, 'new': 1, 'parent': 1}]


In [14]:
import numpy as np

# Construct Nxd array based on the word-sentence frequency
pre_M = []
for word in corpus:
    word_freq = []
    for sentence in sen_count:
        tempfreq = sentence.get(word)
        if tempfreq != None:
            word_freq.append(tempfreq)
        else:
            word_freq.append(0)
    pre_M.append(word_freq)
M = np.array(pre_M)
print(M[0][30:50])

[0 0 0 0 0 0 1 0 2 0 0 0 0 0 0 0 0 0 0 0]


In [15]:
print(M.shape)

(100, 4584)


In [16]:
print(type(M))

<class 'numpy.ndarray'>


In [17]:
def featureVector(target_words: list, pseudowords: list, res: list) -> np.array:
    
    # Create a corpus containing the target words and pseudowords
    corpus = target_words + pseudowords
    
    # Get all the sentences in all documents
    sentences = []
    for doc in res:
        for sen in doc:
            sentences.append(sen)
    
    # Loop through each sentence and count the frequency of each word
    sen_count = []
    for i, sentence in enumerate(sentences):
        counts = dict()
        words = process_document(sentence)
        for word in words:
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
        sen_count.append(counts)
        
    # Construct Nxd array based on the word-sentence frequency
    pre_M = []
    for word in corpus:
        word_freq = []
        for sentence in sen_count:
            tempfreq = sentence.get(word)
            if tempfreq != None:
                word_freq.append(tempfreq)
            else:
                word_freq.append(0)
        pre_M.append(word_freq)
    M = np.array(pre_M)
#     svd
    U, s, V = np.linalg.svd(M)
    M = np.dot(U, np.diag(s))
    
#     norm
    
    M = M / np.linalg.norm(M, axis=1)[:, None]
    
    
    return M


In [18]:
M = featureVector(target_words, pseudowords, res)
print(M)

[[ 2.94517363e-01 -1.09916877e-01  1.09754768e-01 ... -1.19649893e-57
   8.56668817e-57  0.00000000e+00]
 [ 1.86513270e-01  6.82420224e-02  4.01085331e-02 ...  6.39723485e-52
   1.82490552e-51  0.00000000e+00]
 [ 2.86161762e-01 -1.20454719e-01  6.98359679e-02 ...  1.45793383e-51
  -1.34945098e-50  0.00000000e+00]
 ...
 [ 3.26530072e-02 -1.78876092e-02  1.12007960e-02 ...  2.37221014e-42
  -1.44430190e-41  0.00000000e+00]
 [ 1.22920048e-01  2.86932862e-02  3.87232786e-02 ...  1.41558380e-41
  -8.53435940e-41  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]


# Task 1 - Step 4 - input matrix X put them into 50 cluster

In [19]:
# import nltk
# from nltk.cluster import KMeansClusterer

# X = M
# # Set the number of clusters to 50
# num_clusters = 50

# # Create a KMeansClusterer instance with the specified number of clusters
# clusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, avoid_empty_clusters=True)

# # Cluster the words using the feature matrix
# clusters = clusterer.cluster(X, assign_clusters=True)

# # Print the cluster assignments for each word
# for i, cluster in enumerate(clusters):
#     print(f"Word {i+1} is in cluster {cluster}.")

In [20]:
from sklearn.cluster import KMeans

# X = M

# # Set the number of clusters to 50
# num_clusters = 50

# # Create a KMeans instance with the specified number of clusters
# kmeans = KMeans(n_clusters=num_clusters, n_init=10, max_iter=300, tol=1e-04, random_state=0)

# # Cluster the words using the feature matrix
# clusters = kmeans.fit_predict(X)

# visualClusters = {}
# # Print the cluster assignments for each word
# for i, cluster in enumerate(clusters):
# #    print(f"Word {i+1} is in cluster {cluster}.")
#     visualClusters[corpus[i]] = cluster

In [21]:
# def getCluster(X: np.array) -> dict:
#     # Set the number of clusters to 50
#     num_clusters = 50

#     # Create a KMeans instance with the specified number of clusters
#     kmeans = KMeans(n_clusters=num_clusters, n_init=10, max_iter=300, tol=1e-04, random_state=0)

#     # Cluster the words using the feature matrix
#     clusters = kmeans.fit_predict(X)

#     visualClusters = {}
#     # Print the cluster assignments for each word
#     for i, cluster in enumerate(clusters):
#     #    print(f"Word {i+1} is in cluster {cluster}.")
#         visualClusters[corpus[i]] = cluster
#     return visualClusters

In [22]:
def getCluster(X: np.array) -> np.array:
    # Set the number of clusters to 50
    num_clusters = 50
    
    # Create a KMeans instance with the specified number of clusters
    # kmeans = KMeans(n_clusters=num_clusters, n_init=10, max_iter=300, tol=1e-04, random_state=0)
    km = KMeans(n_clusters=50).fit(X)
    
    # Cluster the words using the feature matrix
    # clusters = km.fit_predict(X)
    performance = []
    
    labels = km.labels_
    labels.tolist()
    performance.append((np.sum(labels[0:50] == labels[50:100])) / len(labels[0:50]))
    print("target words: " + str(labels[50:100]))
    print("pesudo words: " + str(labels[0:50]))
    print("Performance: " + str(performance[0]))
        
    return performance

In [23]:
getCluster(M)

target words: [ 3 39 22  9  6  2 43 38  7  4 38  5 28 41  0 13 26 29 16  1 31 19 10 44
 23 20 32 45  1 33 46 35 42 12 17  0 27 37  8 25 15 36 18 11 34 48 30 24
 14 40]
pesudo words: [21 13  6  9  6  2 18 47  7  4 24  5 28 10  0 13 24 29 16  1 47 19 10 23
 23 33  5 28  1 33 21 28 47 12 17 47 27  3 49 25 49  4 18 11 28 11 30 24
 14  9]
Performance: 0.52


[0.52]

In [24]:
print(M.shape)

(100, 100)


In [25]:
clust = getCluster(M)
print(clust)

target words: [49 26 32 10  1  9 40 38 16 17 48  5 13 31 41 22 36  3  6  4 46 30  2 34
 20 29 42  1  4 15 35  1 33 11 27  0 25 44 43 19 14  0  7 23 45 37 12 18
 21 28]
pesudo words: [24 22  1 10  1  9  7 47 16 17 18  5 13  2  8 22 18  3  6  4 47 30  2 20
 20 15  5 13  4 15 24 13 47 11 27 47 25 17 39 19 39 17  7 23 13 23 12 18
 21 10]
Performance: 0.5
[0.5]


# Task 1 - Step 5

In [26]:
# target_cluster = clust[:50]
# pseudo_cluster = clust[50:]
# print(target_cluster,len(target_cluster),"\n",pseudo_cluster, len(pseudo_cluster))

In [27]:
print(pseudowords, len(pseudowords))

['esu', 'enohp', 'eno', 'router', 'ipod', 'camera', 'reyalp', 'teg', 'battery', 'diaper', 'tcudorp', 'work', 'like', 'taerg', 'emit', 'feature', 'melborp', 'good', 'quality', 'zen', 'dluow', 'also', 'sound', 'retupmoc', 'software', 'erutcip', 'llew', 'yllaer', 'micro', 'take', 'ysae', 'gniht', 'neve', 'first', 'used', 'deen', 'creative', 'gab', 'hcum', 'want', 'retteb', 'pmahc', 'mp', 'look', 'og', 'ezis', 'music', 'norton', 'little', 'ecirp'] 50


In [28]:
print(target_words, len(target_words))

['use', 'phone', 'one', 'router', 'ipod', 'camera', 'player', 'get', 'battery', 'diaper', 'product', 'work', 'like', 'great', 'time', 'feature', 'problem', 'good', 'quality', 'zen', 'would', 'also', 'sound', 'computer', 'software', 'picture', 'well', 'really', 'micro', 'take', 'easy', 'thing', 'even', 'first', 'used', 'need', 'creative', 'bag', 'much', 'want', 'better', 'champ', 'mp', 'look', 'go', 'size', 'music', 'norton', 'little', 'price'] 50


In [29]:
# cluster_size = 50
# temp_count = 0
# for i in range(cluster_size):
#     if(target_cluster[i] == pseudo_cluster[i]):
#         temp_count += 1
# p = temp_count/cluster_size
# print(p)

In [30]:
# print(type(clust))

In [31]:
# def getProbability(cluster: np.array) -> int:
#     cluster_size = len(cluster) // 2 
#     temp_count = 0
#     for i in range(cluster_size):
#         if(target_cluster[i] == pseudo_cluster[i]):
#             temp_count += 1
#     p = temp_count/cluster_size
#     return p
# getProbability(clust)